<a href="https://colab.research.google.com/github/martin-fabbri/recsys-pipeline/blob/main/tiktok-pipeline/2_retrieval_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <span style="color:#ff5f27">🧬 Train Retrieval Model </span>


## <span style='color:#ff5f27'> 📝 Install Dependencies </span>

In [2]:
!pip install -qq -U hopsworks[python]
!pip install -qq tensorflow-recommenders

## <span style="color:#ff5f27">📝 Imports </span>

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import StringLookup, Normalization
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
print('tensorflow version:', tf.__version__)
print('tensorflow_recommenders version:', tfrs.__version__)

tensorflow version: 2.17.1
tensorflow_recommenders version: v0.7.3


## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [4]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1196184


## <span style="color:#ff5f27">🔪 Feature Selection </span>


In [5]:
users_fg = fs.get_feature_group(
    name="users",
    version=1,
)

videos_fg = fs.get_feature_group(
    name="videos",
    version=1,
)

interactions_fg = fs.get_feature_group(
    name="interactions",
    version=1,
)

---

In [ ]:
# generic = Generic(locale=Locale.EN)

# obs = pd.DataFrame(
#     [[generic.person.identifier(mask='####-##-####'), 'like', 'WA546S', '3EL60S', 206]],
#     columns=['interaction_id', 'interaction_type', 'user_id', 'video_id', 'watch_time']
# )
# interactions_fg.insert(obs)

,interaction_id,interaction_type,user_id,video_id,watch_time
0,7133-34-1966,like,WA546S,3EL60S,206


---

In [6]:
QUERY_FEATURES = ["user_id", "gender", "age", "country"]
CANDIDATE_FEATURES = ["video_id", "category", "views", "likes", "video_length"]

In [7]:
# Select features for training data
selected_features = interactions_fg.select(["interaction_id"])\
    .join(users_fg.select(QUERY_FEATURES), on="user_id")\
    .join(videos_fg.select(CANDIDATE_FEATURES), on="video_id")

# Uncomment this if you would like to view your selected features
selected_features.show(5)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.96s) 


,interaction_id,user_id,gender,age,country,video_id,category,views,likes,video_length
0,6282-72-3248,NZ031Q,Female,80,Germany,1AP74D,Dance,4036,3127,196
1,4312-89-8518,MJ488B,Other,51,Czech Republic,1NV27Z,Lifestyle,48563,11080,206
2,1008-94-1017,HU937W,Female,21,Micronesia,9WJ38H,Dance,16026,9374,122
3,5269-78-7890,BH602O,Male,86,Martinique,2TQ25J,Cooking,42902,15735,27
4,8122-80-1610,TG318L,Male,72,South Sudan,5OU34O,Technology,417517,364922,54


## <span style="color:#ff5f27">⚙️ Feature View Creation </span>


In [8]:
feature_view = fs.get_or_create_feature_view(
    name='retrieval',
    version=1,
    query=selected_features,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1196184/fs/1183821/fv/retrieval/version/1


## <span style="color:#ff5f27">🏋️ Training Dataset </span>


In [9]:
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=0.1,
    test_size=0.1,
    description='Retrieval dataset splits',
)
train_df.head(3)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (12.11s) 


,interaction_id,user_id,gender,age,country,video_id,category,views,likes,video_length
0,7927-43-0436,VM332Y,Other,13,Somalia,2KD84E,Lifestyle,362035,158609,167
2,5052-59-2238,FT125N,Female,62,Hong Kong SAR China,7LG48D,Lifestyle,3928,3315,11
4,7848-57-6243,RZ535X,Other,51,Fiji,3KA05Q,Dance,208619,83038,200


You will train your retrieval model with a subset of features.

For the query embedding you will use:
- `user_id`: ID of a user.
- `gender`: Gender of a user.
- `age`: age of a user.
- `country`: country if a user.

For the candidate embedding you will use:
- `video_id`: ID of a video.
- `category`: Video Category.
- `views`: Number of video views.
- `likes`: Number of video likes.
- `video_length`: Length of video.


In [10]:
def df_to_ds(df):
    return tf.data.Dataset.from_tensor_slices({col: df[col] for col in df})

BATCH_SIZE = 2048
train_ds = df_to_ds(train_df).batch(BATCH_SIZE).cache().shuffle(BATCH_SIZE*10)
val_ds = df_to_ds(val_df).batch(BATCH_SIZE).cache()

In [11]:
# Query Features
user_id_list = train_df["user_id"].unique().tolist()
countries_list = train_df["country"].unique().tolist()
gender_list = train_df["gender"].unique().tolist()

# Item Features
video_id_list = train_df["video_id"].unique().tolist()
category_list = train_df["category"].unique().tolist()

print(f"⛳️ Number of users: {len(user_id_list)}")
print(f"⛳️ Number of items: {len(video_id_list)}")

⛳️ Number of users: 49932
⛳️ Number of items: 49933


## <span style="color:#ff5f27">🏰 Two Tower Model </span>


In [12]:
EMB_DIM = 32

In [14]:
class QueryTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.emb_dim = EMB_DIM
        self.user_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=user_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # You add an additional embedding to account for unknown tokens.
                len(user_id_list) + 1,
                self.emb_dim
            )
        ])

        self.normalized_age = Normalization(axis=None)

        # Converts strings into integer indices (scikit-learn LabelEncoder analog)
        self.gender_tokenizer = StringLookup(
            vocabulary=gender_list,
            mask_token=None,
        )

        self.country_tokenizer = StringLookup(
            vocabulary=countries_list,
            mask_token=None,
        )

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(self.emb_dim, activation="relu"),
            tf.keras.layers.Dense(self.emb_dim)
        ])

    def call(self, inputs):
        gender_embedding = tf.one_hot(
            self.gender_tokenizer(inputs["gender"]),
            len(gender_list),
        )

        country_embedding = tf.one_hot(
            self.country_tokenizer(inputs["country"]),
            len(countries_list),
        )

        concatenated_inputs = tf.concat([
            self.user_embedding(inputs["user_id"]),
            tf.reshape(self.normalized_age(inputs["age"]), (-1,1)),
            gender_embedding,
            country_embedding,
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs


query_model = QueryTower()

query_model.normalized_age.adapt(train_ds.map(lambda x : x["age"]))

# Initialize model with inputs.
query_df = train_df[QUERY_FEATURES]
query_ds = df_to_ds(query_df).batch(1)
query_model(next(iter(query_ds)))

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot add tensor to the batch: number of elements does not match. Shapes are: [tensor]: [512], [batch]: [2048] [Op:IteratorGetNext] name: 

In [ ]:
class ItemTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.emb_dim = EMB_DIM
        self.video_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=video_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # You add an additional embedding to account for unknown tokens.
                len(video_id_list) + 1,
                self.emb_dim,
            )
        ])

        # Converts strings into integer indices (scikit-learn LabelEncoder analog)
        self.category_tokenizer = StringLookup(
            vocabulary=category_list,
            mask_token=None,
        )

        self.normalized_views = Normalization(axis=None)
        self.normalized_likes = Normalization(axis=None)
        self.normalized_video_length = Normalization(axis=None)

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(self.emb_dim, activation="relu"),
            tf.keras.layers.Dense(self.emb_dim)
        ])

    def call(self, inputs):
        category_embedding = tf.one_hot(
            self.category_tokenizer(inputs["category"]),
            len(category_list),
        )

        concatenated_inputs = tf.concat([
            self.video_embedding(inputs["video_id"]),
            category_embedding,
            tf.reshape(self.normalized_views(inputs["views"]), (-1,1)),
            tf.reshape(self.normalized_views(inputs["likes"]), (-1,1)),
            tf.reshape(self.normalized_views(inputs["video_length"]), (-1,1)),
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs


item_model = ItemTower()

item_df = train_df[CANDIDATE_FEATURES]
item_df.drop_duplicates(subset="video_id", inplace=True)
item_ds = df_to_ds(item_df)

item_model(next(iter(item_ds.batch(1))))

<tf.Tensor: shape=(1, 16), dtype=float32, numpy=
array([[ -8337.219 ,    271.7439,   6766.8276,  -5961.2393,   4387.9194,
         -2594.7898,   -959.5883,  11187.032 ,  -5418.3667,   3178.5222,
           530.5153,  16850.271 ,  -1287.1066,  -2689.3015, -12817.958 ,
          7989.92  ]], dtype=float32)>

In [ ]:
class TwoTowerModel(tf.keras.Model):
    def __init__(self, query_model, item_model):
        super().__init__()
        self.query_model = query_model
        self.item_model = item_model
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=item_ds.batch(BATCH_SIZE).map(self.item_model)
            )
        )

    def train_step(self, batch) -> tf.Tensor:
        # Set up a gradient tape to record gradients.
        with tf.GradientTape() as tape:

            # Loss computation.
            user_embeddings = self.query_model(batch)
            item_embeddings = self.item_model(batch)
            loss = self.task(
                user_embeddings,
                item_embeddings,
                compute_metrics=False,
            )

            # Handle regularization losses as well.
            regularization_loss = sum(self.losses)

            total_loss = loss + regularization_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        metrics = {
            "loss": loss,
            "regularization_loss": regularization_loss,
            "total_loss": total_loss
        }

        return metrics

    def test_step(self, batch) -> tf.Tensor:
        # Loss computation.
        user_embeddings = self.query_model(batch)
        item_embeddings = self.item_model(batch)

        loss = self.task(
            user_embeddings,
            item_embeddings,
            compute_metrics=False,
        )

        # Handle regularization losses as well.
        regularization_loss = sum(self.losses)

        total_loss = loss + regularization_loss

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics

### <span style="color:#ff5f27">🏃🏻‍♂️ Model Training </span>

You'll train our model using the AdamW optimizer, which applies weight regularization during training.

In [ ]:
# Create a TwoTowerModel with the specified query_model and item_model
model = TwoTowerModel(query_model, item_model)

# Define an optimizer using AdamW with a learning rate of 0.01
optimizer = tf.keras.optimizers.AdamW(
    weight_decay=0.001,
    learning_rate=0.01,
)

# Compile the model using the specified optimizer
model.compile(optimizer=optimizer)

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
)

Epoch 1/5
395/395 [==============================] - 27s 62ms/step - loss: 228725.1049 - regularization_loss: 0.0000e+00 - total_loss: 228725.1049 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - val_loss: 4195.9941 - val_regularization_loss: 0.0000e+00 - val_total_loss: 4195.9941
Epoch 2/5
395/395 [==============================] - 25s 62ms/step - loss: 15967.3948 - regularization_loss: 0.0000e+00 - total_loss: 15967.3948 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0

## <span style="color:#ff5f27">🗄️ Upload Model to Model Registry </span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Let's connect to the model registry using the [HSML library](https://docs.hopsworks.ai/machine-learning-api/latest) from Hopsworks.

In [ ]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
class QueryModelModule(tf.Module):
    def __init__(self, query_model):
        self.query_model = query_model

    @tf.function()
    def compute_emb(self, instances):
        query_emb = self.query_model(instances)
        return {
            "user_id": instances["user_id"],
            "gender": instances["gender"],
            "age": instances["age"],
            "country": instances["country"],
            "query_emb": query_emb,
        }

# wrap query_model:   query_model -> query_model_module
query_model = QueryModelModule(model.query_model)

In [ ]:
# Define the input specifications for the instances
instances_spec = {
    'user_id': tf.TensorSpec(shape=(None,), dtype=tf.string, name='user_id'),   # Specification for user IDs
    'gender': tf.TensorSpec(shape=(None,), dtype=tf.string, name='gender'),     # Specification for gender
    'country': tf.TensorSpec(shape=(None,), dtype=tf.string, name='country'),    # Specification for country
    'age': tf.TensorSpec(shape=(None,), dtype=tf.int64, name='age'),              # Specification for age
}

# Get the concrete function for the query_model's compute_emb function using the specified input signatures
signatures = query_model.compute_emb.get_concrete_function(instances_spec)

# Save the query_model along with the concrete function signatures
tf.saved_model.save(
    query_model,           # The model to save
    "query_model",         # Path to save the model
    signatures=signatures, # Concrete function signatures to include
)

2024-10-01 17:22:37,692 INFO: Function `compute_emb` contains input name(s) table_handle, 16213, resource with unsupported characters which will be renamed to query_tower_sequential_string_lookup_none_lookup_lookuptablefindv2_table_handle, query_tower_sequential_embedding_embedding_lookup_16213, query_tower_sequential_1_dense_1_biasadd_readvariableop_resource in the SavedModel.
INFO:tensorflow:Assets written to: query_model/assets
2024-10-01 17:22:42,016 INFO: Assets written to: query_model/assets


In [ ]:
tf.saved_model.save(
    model.item_model,    # The model to save
    "candidate_model",   # Path to save the model
)

2024-10-01 17:22:42,632 WARNING: Skipping full serialization of Keras layer <keras.src.layers.preprocessing.normalization.Normalization object at 0x7f6ccc452050>, because it is not built.
2024-10-01 17:22:42,635 WARNING: Skipping full serialization of Keras layer <keras.src.layers.preprocessing.normalization.Normalization object at 0x7f6cf54f64d0>, because it is not built.
INFO:tensorflow:Assets written to: candidate_model/assets
2024-10-01 17:22:45,419 INFO: Assets written to: candidate_model/assets


In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Infer input schema from data.
query_model_input_schema = Schema(query_df)

# Manually specify output schema.
query_model_output_schema = Schema([{
    "name": "query_embedding",
    "type": "float32",
    "shape": [EMB_DIM],
}])

query_model_schema = ModelSchema(
    input_schema=query_model_input_schema,
    output_schema=query_model_output_schema,
)

query_model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'user_id', 'type': 'object'},
   {'name': 'gender', 'type': 'object'},
   {'name': 'age', 'type': 'int64'},
   {'name': 'country', 'type': 'object'}]},
 'output_schema': {'tensor_schema': [{'name': 'query_embedding',
    'shape': '[16]',
    'type': 'float32'}]}}

In [ ]:
# Sample a query example from the query DataFrame
query_example = query_df.sample().to_dict("records")

# Create a tensorflow model for the query_model in the Model Registry
mr_query_model = mr.tensorflow.create_model(
    name="query_model",                                           # Name of the model
    description="Model that generates query embeddings from user features",  # Description of the model
    input_example=query_example,                                  # Example input for the model
    model_schema=query_model_schema,                              # Schema of the model
)

# Save the query_model to the Model Registry
mr_query_model.save("query_model")                                # Path to save the model

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/562094 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/55 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/576 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1688053 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/78 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/484 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/17565/models/query_model/1


Model(name: 'query_model', version: 1)

In [ ]:
# Define the input schema for the candidate_model based on item_df
candidate_model_input_schema = Schema(item_df)

# Define the output schema for the candidate_model, specifying the shape and type of the output
candidate_model_output_schema = Schema([{
    "name": "candidate_embedding",   # Name of the output feature
    "type": "float32",               # Data type of the output feature
    "shape": [EMB_DIM],              # Shape of the output feature
}])

# Combine the input and output schemas to create the overall model schema for the candidate_model
candidate_model_schema = ModelSchema(
    input_schema=candidate_model_input_schema,    # Input schema for the model
    output_schema=candidate_model_output_schema,  # Output schema for the model
)

# Sample a candidate example from the item DataFrame
candidate_example = item_df.sample().to_dict("records")

# Create a tensorflow model for the candidate_model in the Model Registry
mr_candidate_model = mr.tensorflow.create_model(
    name="candidate_model",                                        # Name of the model
    description="Model that generates candidate embeddings from video features",  # Description of the model
    input_example=candidate_example,                              # Example input for the model
    model_schema=candidate_model_schema,                          # Schema of the model
)

# Save the candidate_model to the Model Registry
mr_candidate_model.save("candidate_model")                        # Path to save the model

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/527309 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/57 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/562 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1671587 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/102 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/563 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/17565/models/candidate_model/1


Model(name: 'candidate_model', version: 1)

---